In [ ]:
%matplotlib inline
!pip install Sastrawi
!pip install swifter
import csv
import sklearn
import swifter
import torch
import ast

import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
from google.colab import files
from wordcloud import WordCloud

from sklearn import svm
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils import resample

from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
from mpl_toolkits.mplot3d import Axes3D

from transformers import AutoTokenizer, AutoModelForSequenceClassification

#LOAD DATA

In [ ]:
import pandas as pd

# Get dataset url from github repository
url = "https://raw.githubusercontent.com/dekapd99/svm-sentiment-analysis/master/dataset.csv"

# Read the dataset with pandas
try:
    df = pd.read_csv(url, delimiter=',')
except pd.errors.ParserError as e:
    print("Error occurred while parsing the CSV file:", e)

df

##Drop Unused Columns

In [ ]:
# Drop the unused Columns
df_dropped = df.drop(['created_at', 'username', 'id', 'id_str', 'quote_count','reply_count',
                      'retweet_count', 'favorite_count', 'geo', 'lang', 'conversation_id',
                      'conversation_id_str', 'media_url_https', 'media_type', 'user_id_str'], axis=1)

# Export the df_dropped DataFrame to a CSV file
df_dropped.to_csv('df_dropped.csv', index=False)

df_dropped

In [ ]:
data = pd.DataFrame(df_dropped[["new_date", "tweet"]])
data

#TEXT PREPROCESSING

#Case Folding

In [ ]:
#Case Folding --> Convert Uppercase Tweet to Lowercase
data["tweet"] = data["tweet"].str.lower()

print("Case Folding Results: \n")
print("================================ \n")
data["tweet"].head(5)

In [ ]:
data.to_csv('df_case_folding.csv', index=False)
data.head(5)

#Data Cleaning

In [ ]:
# Data Cleaning --> Delete Mention Username
def remove_pattern(input_txt, pattern):
  r = re.findall(pattern, input_txt)
  for i in r:
    input_txt = re.sub(i, '', input_txt)
  return input_txt

data["tweet_mention_removed"] = np.vectorize(remove_pattern)(data["tweet"], "@[\w]*")
data

In [ ]:
# Removing Retweet, Number, Punctuation, Emoji / Emotikon, Space, New Line, Hashtag, Link URL
def remove(tweet):
  tweet = re.sub(r'^rt[\s]+', '', tweet)
  tweet = re.sub(r'#', '', tweet)
  tweet = re.sub(r'https\S+', '', tweet)
  tweet = re.sub(r'[0-9]+', '', tweet)
  tweet = re.sub(r'\_', " ", tweet)
  tweet = re.sub('\s+', " ", tweet)
  tweet = re.sub(r"\W+", " ", tweet)
  tweet = re.sub(r'\\n', "", tweet)
  tweet = re.sub(r"\b[a-zA-Z]\b", "", tweet)
  tweet = emoji.replace_emoji(tweet, replace='')
  tweet = re.sub(r':', '', tweet)
  return tweet

def remove_whitespace(tweet):
  return ' '.join(tweet.split())

data["clean_tweet"] = data["tweet_mention_removed"].apply(lambda x: remove(x))

data.head(10)

In [ ]:
data.to_csv('df_clean_tweet.csv', index=False)
data.head(5)

##Tokenization

In [ ]:
# Get dataset url from github repository
url = "https://raw.githubusercontent.com/dekapd99/svm-sentiment-analysis/master/clean_tweet.csv"

# Read the dataset with pandas
try:
    data = pd.read_csv(url, delimiter=',')
except pd.errors.ParserError as e:
    print("Error occurred while parsing the CSV file:", e)

data.head(5)

In [ ]:
# Download nltk resources (only needed once)
nltk.download('punkt')

# Convert clean_tweet values to strings
data['clean_tweet'] = data['clean_tweet'].astype(str)

# Tokenize the clean_tweet column
data['tokens_tweet'] = data['clean_tweet'].apply(lambda x: word_tokenize(x))

# Print the tokenized tweets
print(data['tokens_tweet'])

In [ ]:
data.to_csv('df_token_tweet.csv', index=False)
data.head(5)

##Normalization

In [ ]:
#Normalization
normalizad_word = pd.read_excel("https://raw.githubusercontent.com/dekapd99/svm-sentiment-analysis/master/new_normalisasi.xlsx")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1]

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

data['normalized_tweet'] = data['tokens_tweet'].apply(normalized_term)

data.head(5)

In [ ]:
data.to_csv('df_normalized_tweet.csv', index=False)
data.head(5)

##Stopwords

In [ ]:
factory = StopWordRemoverFactory()
more_stopword = ['yg', 'dg', 'rt', 'dgn', 'ny', 'd', 'klo', 'bapak', 'tingkat', 'buat',
                  'kalo', 'amp', 'biar', 'bikin', 'bilang','ibu', 'jadi', 'laku',
                  'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 'anda', 'terus', 'maksimal',
                  'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 'ente', 'masyarakat',
                  'jd', 'jgn', 'sdh', 'aja', 'n', 't', 'dan', 'sangat', 'moga', 'rakyat',
                  'nyg', 'hehe', 'pen', 'u', 'loh', 'dengan', 'pak', 'bu', 'terus', 'sangat',
                  '&amp', 'yah', 'lagi', 'yang', 'kenapa', 'gitu', 'indonesia', 'polisi republik indonesia',
                  'kamu', 'saya', 'aku', 'untuk', 'jangan', 'hasil', 'republik', 'polri',
                  'dari', 'datang', 'kemana', 'atau', 'lihat', 'sekarang', 'beri',
                  'seperti', 'iya', 'kalau', 'pakai', 'dia', 'doang', 'atas', 'tangan',
                  'mereka', 'pada', 'dan', 'tapi', 'bilang', 'harus', 'apa', 'saja', 'polisi'
                  ]

stopword = factory.create_stop_word_remover()
stopwords = factory.get_stop_words() + more_stopword

In [ ]:
print(stopwords)

In [ ]:
# Apply stopword removal function
data['stopwords_tweet'] = data['normalized_tweet'].apply(lambda tweet: [word for word in tweet if word not in stopwords])

print(data['stopwords_tweet'])
data.head(5)

In [ ]:
data.to_csv('df_stopwords_tweet.csv', index=False)
data.head(5)

##Stemming

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in data['stopwords_tweet']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '

print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])

print(term_dict)
print("------------------------")

# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

data['stemmed_tweet'] = data['stopwords_tweet'].swifter.apply(get_stemmed_term)
print(data['stemmed_tweet'])

In [ ]:
data.to_csv('df_stemmed_tweet.csv', index=False)
data.head(5)

In [ ]:
factory = StopWordRemoverFactory()
more_stopword = ['yg', 'dg', 'rt', 'dgn', 'ny', 'd', 'klo', 'bapak', 'tingkat', 'buat',
                  'kalo', 'amp', 'biar', 'bikin', 'bilang','ibu', 'jadi', 'laku',
                  'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 'anda', 'terus', 'maksimal',
                  'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 'ente', 'masyarakat',
                  'jd', 'jgn', 'sdh', 'aja', 'n', 't', 'dan', 'sangat', 'moga', 'rakyat',
                  'nyg', 'hehe', 'pen', 'u', 'loh', 'dengan', 'pak', 'bu', 'terus', 'sangat',
                  '&amp', 'yah', 'lagi', 'yang', 'kenapa', 'gitu', 'indonesia', 'polisi republik indonesia',
                  'kamu', 'saya', 'aku', 'untuk', 'jangan', 'hasil', 'republik',
                  'dari', 'datang', 'kemana', 'atau', 'lihat', 'sekarang', 'beri',
                  'seperti', 'iya', 'kalau', 'pakai', 'dia', 'doang', 'atas', 'tangan',
                  'mereka', 'pada', 'dan', 'tapi', 'bilang', 'harus', 'apa', 'saja', 'polisi'
                  ]

stopword = factory.create_stop_word_remover()
stopwords = factory.get_stop_words() + more_stopword

# Apply stopword removal function
data['clean'] = data['stemmed_tweet'].apply(lambda tweet: [word for word in tweet if word not in stopwords])

data.to_csv('clean.csv', index=False)

In [ ]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words()
print(stopwords)

#LABELING

In [ ]:
# Get dataset url from GitHub repository
url = "https://raw.githubusercontent.com/dekapd99/svm-sentiment-analysis/master/stemmed_tweet.csv"

# Read the dataset with pandas
try:
    data = pd.read_csv(url, delimiter=',')
except pd.errors.ParserError as e:
    print("Error occurred while parsing the CSV file:", e)

# Convert the string representation of the list to a list of words
data['stemmed_tweet'] = data['stemmed_tweet'].apply(lambda x: ast.literal_eval(x))

# Join the words in 'stemmed_tweet' column into sentences
data['sentence'] = data['stemmed_tweet'].apply(lambda x: ' '.join(x))

# Drop the unused Columns
data = data.drop(['new_date', 'clean_tweet', 'sentiment'], axis=1)

data

In [ ]:
# Load the pre-trained sentiment analysis model for Indonesian
model_name = "indobenchmark/indobert-base-p2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def classify_sentiment(text):
    input_ids = tokenizer.encode(text, add_special_tokens=True)
    input_ids = input_ids[:1024]  # Truncate to maximum length of 512 tokens

    inputs = torch.tensor([input_ids])
    outputs = model(inputs)
    logits = outputs.logits
    predicted_class = logits.argmax(dim=1).item()

    sentiment_labels = ["negatif", "positif"]

    if predicted_class < len(sentiment_labels):
        sentiment = sentiment_labels[predicted_class]
    else:
        sentiment = "Tidak Diketahui"

    return sentiment

# Apply sentiment classification to the "sentence" column
data['sentiment'] = data['sentence'].apply(classify_sentiment)

# Display the updated DataFrame with sentiment values
print(data[['sentence', 'sentiment']])

#VISUALISASI DATA

In [ ]:
# Get dataset url from github repository
url = "https://raw.githubusercontent.com/dekapd99/svm-sentiment-analysis/master/labeling.csv"

# Read the dataset with pandas
try:
    data = pd.read_csv(url, delimiter=',')
except pd.errors.ParserError as e:
    print("Error occurred while parsing the CSV file:", e)

data

#Value Count untuk Seluruh Sentiment

In [ ]:
sentiment_counts = data['sentiment'].value_counts()

# Calculate the sum based on the counts
total_sum = sentiment_counts.sum()

# Display the counts & sum
print(sentiment_counts)
print("Total:", total_sum)

##Distribusi Kelas Negatif dan Positif dengan Barchart

In [ ]:
# Count the occurrences of each sentiment value
sentiment_counts = data['sentiment'].value_counts()

# Create the bar chart
plt.bar(sentiment_counts.index, sentiment_counts.values)

# Set the tick labels for x-axis
plt.xticks(sentiment_counts.index, ['Positif', 'Negatif'])

# Set the title and labels
plt.title("Barchart Sentimen")
plt.xlabel("Sentiment")
plt.ylabel("Count")

# Show the plot
plt.show()

##Distribusi Kelas Negatif dan Positif dengan Piechart

In [ ]:
# Count the occurrences of each sentiment value
sentiment_counts = data['sentiment'].value_counts()

# Create the pie chart
plt.pie(sentiment_counts, autopct='%.2f')

# Set the title and legend
plt.title("PieChart Sentiment")
plt.legend(sentiment_counts.index)

# Show the plot
plt.show()

##Word Cloud

In [ ]:
plt.figure(figsize=(10, 10))
# Convert float values to strings
sentence_texts = data['sentence'].astype(str)
wc = WordCloud(max_words=1000, min_font_size=10, height=600, width=1200, collocations=False,
               background_color='#e6ffed', colormap='summer').generate(' '.join(sentence_texts))

plt.axis('off')
plt.imshow(wc)

##Word Cloud Positif

In [ ]:
# Get dataset url from github repository
url = "https://raw.githubusercontent.com/dekapd99/svm-sentiment-analysis/master/labeling-positif.csv"

# Read the dataset with pandas
try:
    data_positif = pd.read_csv(url, delimiter=',')
except pd.errors.ParserError as e:
    print("Error occurred while parsing the CSV file:", e)

plt.figure(figsize=(10, 10))
# Convert float values to strings
positif_sentence = data_positif['sentence'].astype(str)
wc_positif = WordCloud(max_words=1000, min_font_size=10, height=600, width=1200, collocations=True,
               background_color='#e6ffed', colormap='summer').generate(' '.join(positif_sentence))

plt.axis('off')
plt.imshow(wc_positif)

# Join all sentences into a single string
positif_text = ' '.join(positif_sentence)

# Generate word frequency count
positif_word_counts = pd.Series(positif_text.split()).value_counts()

# Display the word frequency count
print(positif_word_counts.head(10))

##Word Cloud Negatif

In [ ]:
# Get dataset url from github repository
url = "https://raw.githubusercontent.com/dekapd99/svm-sentiment-analysis/master/labeling-negatif.csv"

# Read the dataset with pandas
try:
    data_negatif = pd.read_csv(url, delimiter=',')
except pd.errors.ParserError as e:
    print("Error occurred while parsing the CSV file:", e)

plt.figure(figsize=(10, 10))

# Convert float values to strings
negatif_sentence = data_negatif['sentence'].astype(str)
wc_negatif = WordCloud(max_words=1000, min_font_size=10, height=600, width=1200, collocations=True,
               background_color='#e6ffed', colormap='autumn').generate(' '.join(negatif_sentence))

plt.axis('off')
plt.imshow(wc_negatif)

# Join all sentences into a single string
negatif_text = ' '.join(negatif_sentence)

# Generate word frequency count
negatif_word_counts = pd.Series(negatif_text.split()).value_counts()

# Display the word frequency count
print(negatif_word_counts.head(10))

## 3D Visualisasi

In [ ]:
# Membuat data sentimen positif dan negatif
n_samples_positif = 3224
n_samples_negatif = 1141

# Menghasilkan data sentimen positif
mean_positif = [1, 1, 1]
cov_positif = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
positive_samples = np.random.multivariate_normal(mean_positif, cov_positif, n_samples_positif)

# Menghasilkan data sentimen negatif
mean_negatif = [-1, -1, -1]
cov_negatif = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
negative_samples = np.random.multivariate_normal(mean_negatif, cov_negatif, n_samples_negatif)

# Membuat subplot 3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot data sentimen positif
ax.scatter(positive_samples[:, 0], positive_samples[:, 1], positive_samples[:, 2], c='blue', label='Positif')

# Plot data sentimen negatif
ax.scatter(negative_samples[:, 0], negative_samples[:, 1], negative_samples[:, 2], c='red', label='Negatif')

# Konfigurasi label sumbu
ax.set_xlabel('Dimensi 1')
ax.set_ylabel('Dimensi 2')
ax.set_zlabel('Dimensi 3')

# Tampilkan legenda
ax.legend()

# Tampilkan plot
plt.show()

#Pembuatan Model dan Pengujian Hasil

In [ ]:
# Get dataset url from github repository
url = "https://raw.githubusercontent.com/dekapd99/svm-sentiment-analysis/master/labeling.csv"

# Read the dataset with pandas
try:
    data = pd.read_csv(url, delimiter=',')
except pd.errors.ParserError as e:
    print("Error occurred while parsing the CSV file:", e)

data

##Oversampling Data dengan Duplikasi

In [ ]:
# Mengekstrak sampel negatif dan positif
negative_samples = data[data['sentiment'] == 'negatif']
positive_samples = data[data['sentiment'] == 'positif']

# Oversampling sampel negatif
oversampled_negative = resample(negative_samples,
                                replace=True,  # Mengizinkan duplikasi
                                n_samples=len(positive_samples),  # Jumlah sampel yang sama dengan sampel positif
                                random_state=42)

# Menggabungkan kembali sampel positif dengan sampel negatif yang sudah di-oversampling
oversampled_data = pd.concat([positive_samples, oversampled_negative])

# Count the number of samples in the "negatif" and "positif" classes
count_negatif = len(oversampled_data[oversampled_data['sentiment'] == 'negatif'])
count_positif = len(oversampled_data[oversampled_data['sentiment'] == 'positif'])

# Print the counts
print("Jumlah sampel negatif:", count_negatif)
print("Jumlah sampel positif:", count_positif)

In [ ]:
# Count the occurrences of each sentiment value
sentiment_counts = oversampled_data['sentiment'].value_counts()

# Create the bar chart
plt.bar(sentiment_counts.index, sentiment_counts.values)

# Set the tick labels for x-axis
plt.xticks(sentiment_counts.index, ['Positif', 'Negatif'])

# Set the title and labels
plt.title("Barchart Sentimen")
plt.xlabel("Sentiment")
plt.ylabel("Count")

# Show the plot
plt.show()

##TF-IDF Vectorizer

In [ ]:
# Create TF-IDF vectorizer
oversampled_data.dropna(subset=['sentence'], inplace=True)

tfidfconverter = TfidfVectorizer(max_features=None,
                                 ngram_range=(1,2), analyzer="word", min_df=0, max_df=0.9,
                                 smooth_idf=True, use_idf=True)
X = tfidfconverter.fit_transform(oversampled_data['sentence']).toarray()

print(X)

##Skenario Pengujian 60:40

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, oversampled_data['sentiment'],
                                                    test_size=0.40, random_state=42)

In [ ]:
print("X_train count:", X_train.shape[0])
print("X_test count:", X_test.shape[0])
print("y_train count:", y_train.shape[0])
print("y_test count:", y_test.shape[0])

In [ ]:
model_svm = SVC(kernel='rbf', random_state=42)
model_svm.fit(X_train, y_train)
predicted_svm = model_svm.predict(X_test)

cm = confusion_matrix(y_test, predicted_svm)
print("Confusion Matrix:")
print(cm)

print("Classification Report:")
print(classification_report(y_test, predicted_svm))

accuracy = accuracy_score(y_test, predicted_svm)
print("Accuracy:", accuracy * 100)

In [ ]:
import seaborn as sns
confusionmatrix = confusion_matrix(y_test, predicted_svm)
f, ax = plt.subplots(figsize =(5,5))
sns.heatmap(confusionmatrix,annot = True,linewidths=0.5,linecolor="red",fmt = ".0f",ax=ax)
plt.xlabel("predicted_svm")
plt.ylabel("y_test")
plt.show()

##Skenario Pengujian 70:30

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, oversampled_data['sentiment'],
                                                    test_size=0.30, random_state=42)

In [ ]:
print("X_train count:", X_train.shape[0])
print("X_test count:", X_test.shape[0])
print("y_train count:", y_train.shape[0])
print("y_test count:", y_test.shape[0])

In [ ]:
model_svm = SVC(kernel='rbf', random_state=42)
model_svm.fit(X_train, y_train)
predicted_svm = model_svm.predict(X_test)

cm = confusion_matrix(y_test, predicted_svm)
print("Confusion Matrix:")
print(cm)

print("Classification Report:")
print(classification_report(y_test, predicted_svm))

accuracy = accuracy_score(y_test, predicted_svm)
print("Accuracy:", accuracy * 100)

In [ ]:
import seaborn as sns
confusionmatrix = confusion_matrix(y_test, predicted_svm)
f, ax = plt.subplots(figsize =(5,5))
sns.heatmap(confusionmatrix,annot = True,linewidths=0.5,linecolor="red",fmt = ".0f",ax=ax)
plt.xlabel("predicted_svm")
plt.ylabel("y_test")
plt.show()

#Skenario Pengujian 80:20

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, oversampled_data['sentiment'],
                                                    test_size=0.30, random_state=42)

In [ ]:
print("X_train count:", X_train.shape[0])
print("X_test count:", X_test.shape[0])
print("y_train count:", y_train.shape[0])
print("y_test count:", y_test.shape[0])

In [ ]:
model_svm = SVC(kernel='rbf', random_state=42)
model_svm.fit(X_train, y_train)
predicted_svm = model_svm.predict(X_test)

cm = confusion_matrix(y_test, predicted_svm)
print("Confusion Matrix:")
print(cm)

print("Classification Report:")
print(classification_report(y_test, predicted_svm))

accuracy = accuracy_score(y_test, predicted_svm)
print("Accuracy:", accuracy * 100)

In [ ]:
import seaborn as sns
confusionmatrix = confusion_matrix(y_test, predicted_svm)
f, ax = plt.subplots(figsize =(5,5))
sns.heatmap(confusionmatrix,annot = True,linewidths=0.5,linecolor="red",fmt = ".0f",ax=ax)
plt.xlabel("predicted_svm")
plt.ylabel("y_test")
plt.show()